### Normalization script already executed in jupyter and the code and output screenshot pasted in word

In [7]:
import sys, time, re, subprocess
from urllib.parse import quote_plus

# ------------------ CONFIG (edit) ------------------
MYSQL_USER = "root"
MYSQL_PASS = "pass123"
MYSQL_HOST = "127.0.0.1"
MYSQL_PORT = 3306
MYSQL_DB   = "job_trends"
RAW_TABLE  = "jobs_in_data"
CHUNKSIZE  = 20000
JOBS_BATCH = 1000
DEMO_MODE  = True   # <<=== ✅ Set True for safe demo (no DB writes)
# ---------------------------------------------------

def ensure(pkg):
    try:
        __import__(pkg)
    except ModuleNotFoundError:
        print(f"Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])

for p in ("pymysql","sqlalchemy","pandas"):
    ensure(p)

import pandas as pd
from sqlalchemy import create_engine, text
import sqlalchemy
import pymysql
from sqlalchemy.exc import IntegrityError

print("Python:", sys.executable)
print("pymysql:", pymysql.__version__)
print("sqlalchemy:", sqlalchemy.__version__)
print("pandas:", pd.__version__)

# Build DB engine
db_url = f"mysql+pymysql://{MYSQL_USER}:{quote_plus(MYSQL_PASS)}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}?charset=utf8mb4"
engine = create_engine(db_url, pool_recycle=3600, pool_pre_ping=True)

# Test connection
try:
    with engine.connect() as conn:
        print("DB reachable (SELECT 1):", conn.execute(text("SELECT 1")).scalar())
except Exception as e:
    print("❌ Cannot reach DB; fix connection first.")
    raise

# Create tables if needed (demo-safe)
if not DEMO_MODE:
    create_sql = """
    CREATE TABLE IF NOT EXISTS companies (
      Company_ID INT PRIMARY KEY,
      Company_Name VARCHAR(255) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

    CREATE TABLE IF NOT EXISTS locations (
      Location_ID INT PRIMARY KEY,
      Location VARCHAR(255) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

    CREATE TABLE IF NOT EXISTS industries (
      Industry_ID INT PRIMARY KEY,
      Industry VARCHAR(255) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

    CREATE TABLE IF NOT EXISTS skills (
      Skill_ID INT PRIMARY KEY,
      Skill_Name VARCHAR(255) UNIQUE
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

    CREATE TABLE IF NOT EXISTS jobs (
      Job_ID INT PRIMARY KEY,
      Job_Title VARCHAR(255),
      Job_Description TEXT,
      Company_ID INT,
      Location_ID INT,
      Industry_ID INT,
      Salary_USD VARCHAR(100),
      Employment_Type VARCHAR(100),
      Experience_Level VARCHAR(100),
      Work_Setting VARCHAR(100)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;

    CREATE TABLE IF NOT EXISTS job_skills (
      Job_ID INT,
      Skill_ID INT,
      PRIMARY KEY (Job_ID, Skill_ID)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    with engine.begin() as conn:
        for stmt in create_sql.strip().split(';'):
            stmt = stmt.strip()
            if stmt:
                conn.execute(text(stmt))
    print("Ensured normalized tables exist.")
else:
    print("Skipping table creation/writes- already exisits.")

# Load lookup metadata
with engine.begin() as conn:
    max_job_id = int(conn.execute(text("SELECT COALESCE(MAX(Job_ID), 0) FROM jobs")).scalar() or 0)
    max_company_id = int(conn.execute(text("SELECT COALESCE(MAX(Company_ID), 0) FROM companies")).scalar() or 0)
    max_location_id = int(conn.execute(text("SELECT COALESCE(MAX(Location_ID), 0) FROM locations")).scalar() or 0)
    max_industry_id = int(conn.execute(text("SELECT COALESCE(MAX(Industry_ID), 0) FROM industries")).scalar() or 0)
    max_skill_id = int(conn.execute(text("SELECT COALESCE(MAX(Skill_ID), 0) FROM skills")).scalar() or 0)

    existing_companies = {r[1]: r[0] for r in conn.execute(text("SELECT Company_ID, Company_Name FROM companies")).fetchall()}
    existing_locations = {r[1]: r[0] for r in conn.execute(text("SELECT Location_ID, Location FROM locations")).fetchall()}
    existing_industries = {r[1]: r[0] for r in conn.execute(text("SELECT Industry_ID, Industry FROM industries")).fetchall()}
    existing_skills = {r[1]: r[0] for r in conn.execute(text("SELECT Skill_ID, Skill_Name FROM skills")).fetchall()}

print("Loaded existing data.")
print(dict(
    max_job_id=max_job_id, max_company_id=max_company_id,
    max_location_id=max_location_id, max_industry_id=max_industry_id,
    max_skill_id=max_skill_id
))

companies_map = dict(existing_companies)
locations_map = dict(existing_locations)
industries_map = dict(existing_industries)
skills_map = dict(existing_skills)

next_company_id = max_company_id + 1
next_location_id = max_location_id + 1
next_industry_id = max_industry_id + 1
next_skill_id = max_skill_id + 1
next_job_id = max_job_id + 1

print("Next IDs:", dict(
    next_job_id=next_job_id,
    next_company_id=next_company_id,
    next_location_id=next_location_id,
    next_industry_id=next_industry_id,
    next_skill_id=next_skill_id
))

def find_col(cols, candidates):
    m = {c.lower(): c for c in cols}
    for n in candidates:
        if n.lower() in m:
            return m[n.lower()]
    return None

def flush_jobs(engine, jobs_batch, job_skills_batch):
    if DEMO_MODE:
        print(f" flush {len(jobs_batch)} jobs and {len(job_skills_batch)} job_skills")
        jobs_batch.clear(); job_skills_batch.clear()
        return
    if not jobs_batch and not job_skills_batch:
        return
    with engine.begin() as conn:
        if jobs_batch:
            dfj = pd.DataFrame(jobs_batch)
            dfj.to_sql('jobs', con=conn, if_exists='append', index=False, method='multi')
        if job_skills_batch:
            dfjs = pd.DataFrame(job_skills_batch)
            dfjs.to_sql('job_skills', con=conn, if_exists='append', index=False, method='multi')
    print(f"Flushed {len(jobs_batch)} jobs and {len(job_skills_batch)} job_skills.")
    jobs_batch.clear(); job_skills_batch.clear()

# Read & simulate normalization
reader = pd.read_sql(f"SELECT * FROM `{RAW_TABLE}` LIMIT 200", engine)  # smaller read for demo
print(f"Loaded {len(reader)} sample rows for demo.")
reader = reader.dropna(how='all').copy()
reader.columns = [c.strip() for c in reader.columns]

job_col = find_col(reader.columns, ["job_title","title"])
desc_col = find_col(reader.columns, ["job_description","description"])
company_col = find_col(reader.columns, ["company","company_name"])
location_col = find_col(reader.columns, ["location","company_location"])
industry_col = find_col(reader.columns, ["industry","sector"])
skills_col = find_col(reader.columns, ["skills","key_skills"])

jobs_batch = []; job_skills_batch = []
processed = 0
start_time = time.time()

for idx, row in reader.iterrows():
    jt = row.get(job_col)
    jd = row.get(desc_col)
    cname = row.get(company_col)
    loc = row.get(location_col)
    ind = row.get(industry_col)
    sks = row.get(skills_col)

    if cname and cname not in companies_map:
        companies_map[cname] = next_company_id; next_company_id += 1
    if loc and loc not in locations_map:
        locations_map[loc] = next_location_id; next_location_id += 1
    if ind and ind not in industries_map:
        industries_map[ind] = next_industry_id; next_industry_id += 1

    jid = next_job_id; next_job_id += 1
    jobs_batch.append({
        "Job_ID": jid, "Job_Title": jt, "Job_Description": jd,
        "Company_ID": companies_map.get(cname),
        "Location_ID": locations_map.get(loc),
        "Industry_ID": industries_map.get(ind),
        "Salary_USD": None, "Employment_Type": None,
        "Experience_Level": None, "Work_Setting": None
    })

    if sks:
        skills_found = re.split(r"[;,|\n]+", str(sks))
        for sk in [s.strip() for s in skills_found if s.strip()]:
            if sk not in skills_map:
                skills_map[sk] = next_skill_id; next_skill_id += 1
            job_skills_batch.append({"Job_ID": jid, "Skill_ID": skills_map[sk]})

    processed += 1
    if len(jobs_batch) >= JOBS_BATCH:
        flush_jobs(engine, jobs_batch, job_skills_batch)

flush_jobs(engine, jobs_batch, job_skills_batch)
print(f"\n✅ Processed {processed} rows in {(time.time()-start_time):.1f}s")

# Skip DB overwrites
if DEMO_MODE:
    print("\n Skipping writing lookup tables (companies, locations, industries, skills).")
else:
    with engine.begin() as conn:
        pd.DataFrame([{"Company_ID": v, "Company_Name": k} for k, v in companies_map.items()]).to_sql('companies', con=conn, if_exists='replace', index=False)
        pd.DataFrame([{"Location_ID": v, "Location": k} for k, v in locations_map.items()]).to_sql('locations', con=conn, if_exists='replace', index=False)
        pd.DataFrame([{"Industry_ID": v, "Industry": k} for k, v in industries_map.items()]).to_sql('industries', con=conn, if_exists='replace', index=False)
        pd.DataFrame([{"Skill_ID": v, "Skill_Name": k} for k, v in skills_map.items()]).to_sql('skills', con=conn, if_exists='replace', index=False)
    print("Wrote lookup tables safely.")

print("\n✅ Normalization demo complete.")
print("Companies:", len(companies_map), 
      "Locations:", len(locations_map), 
      "Industries:", len(industries_map),
      "Skills:", len(skills_map))


Python: c:\Users\HP\Downloads\job-trends-app\job-trends-app\.venv\Scripts\python.exe
pymysql: 1.4.6
sqlalchemy: 2.0.44
pandas: 2.3.3
DB reachable (SELECT 1): 1
Skipping table creation/writes- already exisits.
Loaded existing data.
{'max_job_id': 19472, 'max_company_id': 0, 'max_location_id': 42, 'max_industry_id': 0, 'max_skill_id': 0}
Next IDs: {'next_job_id': 19473, 'next_company_id': 1, 'next_location_id': 43, 'next_industry_id': 1, 'next_skill_id': 1}
Loaded 200 sample rows for demo.
 flush 200 jobs and 0 job_skills

✅ Processed 200 rows in 0.0s

 Skipping writing lookup tables (companies, locations, industries, skills).

✅ Normalization demo complete.
Companies: 0 Locations: 42 Industries: 0 Skills: 0
